In [3]:
import numpy as np
import os
import tensorflow as tf

################################ 이미지 다운로드 ########################################
# 분석하기 전에 파일 시스템 경로(../.../.../...)를 입출력하기 위해 사용
import pathlib

dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(origin=dataset_url, 
                                   fname='flower_photos', 
                                   untar=True)
data_dir = pathlib.Path(data_dir)

# 이미지 개수 출력
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)


228818944/228813984 [==============================] - 11s 0us/step
3670


In [8]:
# 장미꽃 출력
roses = list(data_dir.glob('roses/*'))

print(data_dir)

C:\Users\whals\.keras\datasets\flower_photos


In [51]:
##################################### 전처리기 ##############################################

np.random.seed(3)
tf.random.set_seed(3)

# 분류 대상 카테고리 선택하기
caltech_dir = "C:/Users/whals/.keras/datasets/flower_photos"
categories = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]
nb_classes = len(categories)  # 5

#lists = list(data_dir.glob('*/*.jpg'))


# 이미지 크기 지정 (64 X 64)
#pixels = image_w * image_h * 3
image_w = 64
image_h = 64
batch_size = 30

# 이미지 데이터 읽어 들이기
all_image_paths = []
all_onehot_labels = []

for idx, cat in enumerate(categories):
    # 레이블 지정
    label = [0 for i in range(nb_classes)]  # one-hot준비 [0,0,0,0,0]
    label[idx] = 1   # one-hot 리스트 생성
    # 이미지
    image_dir = caltech_dir + "/" + cat
    # 각 폴더에 있는 모든 파일이름에 대한 리스트 생성
    files = glob.glob(image_dir+"/*.jpg")
    for f in files:
        all_image_paths.append(f)
        all_onehot_labels.append(label)

# tf.iamge형태로 이미지 로딩
# jpg를 디코딩하고 사이즈 조절과 정규화를 동시 진행
# label인자에 대한 처리는 하지 않고 단순히 받아서 그대로 리턴함
def load_image_path_label(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [image_w,image_h])
    image /= 255.0  # normalize to [0,1] range
    return image, label

full_dataset = tf.data.Dataset.from_tensor_slices((all_image_paths, all_onehot_labels))
full_dataset = full_dataset.map(load_image_path_label)

# 전체 데이터 갯수 계산
DATASET_SIZE = len(all_image_paths)

# 3:1학습과 테스트 데이터 분리
train_size = int(0.75 * DATASET_SIZE)
test_size = DATASET_SIZE - train_size

# 랜덤하게 shuffling
full_dataset = full_dataset.shuffle(buffer_size = int(DATASET_SIZE*1.5))


# 학습 데이터 생성
train_ds = full_dataset.take(train_size)
train_ds = train_ds.batch(batch_size)

# 나머지를 테스트 용으로 사용
test_ds = full_dataset.skip(train_size)
test_ds = test_ds.batch(batch_size)

########################### 모델 생성 ###################################
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size=(3, 3), input_shape=(image_w, image_h, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2)) # default
model.add(Dropout(0.3))

model.add(Conv2D(filters = 64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))      
model.add(Dropout(0.3))


model.add(Conv2D(filters = 128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(625, activation='relu'))
model.add(Dropout(0.3))

# softmax
model.add(Dense(nb_classes, activation='softmax'))

#optimizer = "Adam"        
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
  
model.summary()

model.fit(train_ds, batch_size=batch_size, epochs = 35)

model.evaluate(test_ds)

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_54 (Conv2D)           (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_54 (MaxPooling (None, 31, 31, 32)        0         
_________________________________________________________________
dropout_72 (Dropout)         (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_55 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_73 (Dropout)         (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 12, 12, 128)     

[0.10704797506332397, 0.9738562107086182]

In [50]:

##################################### 전처리기

np.random.seed(3)
tf.random.set_seed(3)

# 분류 대상 카테고리 선택하기
caltech_dir = "C:/Users/whals/.keras/datasets/flower_photos"
categories = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]
nb_classes = len(categories)  # 5

lists = list(data_dir.glob('*/*.jpg'))


# 이미지 크기 지정 (64 X 64)
#pixels = image_w * image_h * 3
image_w = 64
image_h = 64
batch_size = 30

# 이미지 데이터 읽어 들이기
all_image_paths = []
all_onehot_labels = []

for idx, cat in enumerate(categories):
    # 레이블 지정
    label = [0 for i in range(nb_classes)]  # one-hot준비 [0,0,0,0,0]
    label[idx] = 1   # one-hot 리스트 생성
    # 이미지
    image_dir = caltech_dir + "/" + cat
    # 각 폴더에 있는 모든 파일이름에 대한 리스트 생성
    files = glob.glob(image_dir+"/*.jpg")
    for f in files:
        all_image_paths.append(f)
        all_onehot_labels.append(label)

# tf.iamge형태로 이미지 로딩
# jpg를 디코딩하고 사이즈 조절과 정규화를 동시 진행
# label인자에 대한 처리는 하지 않고 단순히 받아서 그대로 리턴함
def load_image_path_label(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [image_w,image_h])
    image /= 255.0  # normalize to [0,1] range
    return image, label

full_dataset = tf.data.Dataset.from_tensor_slices((all_image_paths, all_onehot_labels))
full_dataset = full_dataset.map(load_image_path_label)

# 전체 데이터 갯수 계산
DATASET_SIZE = len(all_image_paths)

# 3:1학습과 테스트 데이터 분리
train_size = int(0.75 * DATASET_SIZE)
test_size = DATASET_SIZE - train_size
# 랜덤하게 shuffling
full_dataset = full_dataset.shuffle(buffer_size = int(DATASET_SIZE*1.5))


# 학습 데이터 생성
train_ds = full_dataset.take(train_size)
train_ds = train_ds.batch(30)

# 나머지를 테스트 용으로 사용
test_ds = full_dataset.skip(train_size)
test_ds = test_ds.batch(30)

########################### 모델 생성 ###################################
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size=(3, 3), input_shape=(image_w, image_h, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2)) # default
model.add(Dropout(0.3))

model.add(Conv2D(filters = 64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))      
model.add(Dropout(0.3))


model.add(Conv2D(filters = 128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(625, activation='relu'))
model.add(Dropout(0.3))

# softmax
model.add(Dense(nb_classes, activation='softmax'))

#Adam          
model.compile(optimizer = "RMSProp",  # optimizer=tf.keras.optimizers.Adam(0.001)
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
  
model.summary()

model.fit(train_ds, batch_size=batch_size, epochs = 30)

model.evaluate(test_ds)

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_51 (Conv2D)           (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_51 (MaxPooling (None, 31, 31, 32)        0         
_________________________________________________________________
dropout_68 (Dropout)         (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_52 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_69 (Dropout)         (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 12, 12, 128)     

[0.10813818126916885, 0.9694989323616028]